In [10]:
import polars as pl


nombre_archivo = "competencia_02_crudo.csv.gz"
nombre_archivo_final = "competencia_02_con_clase.csv.gz"
df = pl.read_csv(
    nombre_archivo,
    infer_schema_length=100000  # mirar más filas para inferir (con 10000 fallaba)
)

print(df.shape)

print(df['foto_mes'].value_counts())

# 2. Función para agregar solo clase_ternaria
def agregar_clase_ternaria_optimizada(df: pl.DataFrame) -> pl.DataFrame:
    """
    Versión optimizada usando Polars.
    Solo devuelve clase_ternaria sin dejar columnas intermedias.
    """
    max_foto_mes = df["foto_mes"].max()

    return df.lazy().sort(["numero_de_cliente", "foto_mes"]).with_columns([
        pl.when(
            (pl.col("foto_mes").shift(-1).over("numero_de_cliente").is_null()) & 
            (pl.col("foto_mes") != max_foto_mes)
        ).then(pl.lit("BAJA+1"))
        .when(
            (pl.col("foto_mes").shift(-2).over("numero_de_cliente").is_null()) & 
            (pl.col("foto_mes") <= max_foto_mes - 2)
        ).then(pl.lit("BAJA+2"))
        .otherwise(pl.lit("CONTINUA"))
        .alias("clase_ternaria")
    ]).collect()

# 3. Agregar la columna clase_ternaria al dataframe
df = agregar_clase_ternaria_optimizada(df)

# 4. Tabla de frecuencia de clase_ternaria por foto_mes
tabla = (
    df.pivot(
        values="numero_de_cliente",
        index="foto_mes",
        on="clase_ternaria",
        aggregate_function="len"
    )
    .fill_null(0)
)

print(tabla)

print(df.shape)
df.write_csv(nombre_archivo_final)
print("Guuuuuuuuuuuuuuuuuaaaaaaaaaaaaaaaaaaardo")

(4717958, 154)
shape: (32, 2)
┌──────────┬────────┐
│ foto_mes ┆ count  │
│ ---      ┆ ---    │
│ i64      ┆ u32    │
╞══════════╪════════╡
│ 201901   ┆ 124273 │
│ 201908   ┆ 132143 │
│ 202107   ┆ 164596 │
│ 201909   ┆ 133757 │
│ 202011   ┆ 160250 │
│ …        ┆ …      │
│ 201902   ┆ 125401 │
│ 202001   ┆ 143464 │
│ 202101   ┆ 161527 │
│ 202008   ┆ 156461 │
│ 202103   ┆ 163109 │
└──────────┴────────┘
shape: (32, 4)
┌──────────┬──────────┬────────┬────────┐
│ foto_mes ┆ CONTINUA ┆ BAJA+2 ┆ BAJA+1 │
│ ---      ┆ ---      ┆ ---    ┆ ---    │
│ i64      ┆ u32      ┆ u32    ┆ u32    │
╞══════════╪══════════╪════════╪════════╡
│ 201901   ┆ 122918   ┆ 720    ┆ 635    │
│ 201902   ┆ 123985   ┆ 693    ┆ 723    │
│ 201903   ┆ 124535   ┆ 738    ┆ 694    │
│ 201904   ┆ 125293   ┆ 502    ┆ 743    │
│ 201905   ┆ 126016   ┆ 681    ┆ 505    │
│ …        ┆ …        ┆ …      ┆ …      │
│ 202104   ┆ 161340   ┆ 1126   ┆ 952    │
│ 202105   ┆ 161946   ┆ 842    ┆ 1129   │
│ 202106   ┆ 162336   ┆ 1135   ┆ 84